In [1]:
import healpy as hp
import pyccl as ccl

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Button, Slider
import ipympl
import IPython
from scipy import stats
import itertools
import time

In [2]:
def bin_mat(r=[],mat=[],r_bins=[]):
    bin_center=0.5*(r_bins[1:]+r_bins[:-1])
    n_bins=len(bin_center)
    ndim=len(mat.shape)
    mat_int=np.zeros([n_bins]*ndim,dtype='float64')
    norm_int=np.zeros([n_bins]*ndim,dtype='float64')
    bin_idx=np.digitize(r,r_bins)-1
    r2=np.sort(np.unique(np.append(r,r_bins))) #this takes care of problems around bin edges
    dr=np.gradient(r2)
    r2_idx=[i for i in np.arange(len(r2)) if r2[i] in r]
    dr=dr[r2_idx]
    r_dr=r*dr

    ls=['i','j','k','l']
    s1=ls[0]
    s2=ls[0]
    r_dr_m=r_dr
    for i in np.arange(ndim-1):
        s1=s2+','+ls[i+1]
        s2+=ls[i+1]
        r_dr_m=np.einsum(s1+'->'+s2,r_dr_m,r_dr)#works ok for 2-d case

    mat_r_dr=mat*r_dr_m
    for indxs in itertools.product(np.arange(min(bin_idx),n_bins),repeat=ndim):
        # x={} #np.zeros_like(mat_r_dr,dtype='bool')
        norm_ijk=1
        mat_t=[]
        for nd in np.arange(ndim):
            slc = [slice(None)] * (ndim)
            #x[nd]=bin_idx==indxs[nd]
            slc[nd]=bin_idx==indxs[nd]
            if nd==0:
                mat_t=mat_r_dr[slc[0]]
            else:
                mat_t=mat_t[slc[0]]
            norm_ijk*=np.sum(r_dr[slc[nd]])
        if norm_ijk==0:
            continue
        mat_int[indxs]=np.sum(mat_t)/norm_ijk
        norm_int[indxs]=norm_ijk
    return bin_center,mat_int

In [3]:
def create_bin(min_ell, max_ell, n, cgg, ckg, ckk):
    x = np.arange(min_ell, max_ell, 1)
    bin_edges = np.linspace(np.min(x), np.max(x), n)
    bin_centers, galaxy_galaxy_bin = bin_mat(x, np.array(cgg), bin_edges)
    _, lensing_galaxy_bin = bin_mat(x, np.array(ckg), bin_edges)
    _, lensing_lensing_bin = bin_mat(x, np.array(ckk), bin_edges)
    return bin_centers, galaxy_galaxy_bin, lensing_galaxy_bin,lensing_lensing_bin

In [4]:
def default_nz(z):
  return np.exp(-0.5*((z - 0.5)/0.1)**2)

def default_b1(z):
  return np.ones_like(z)

In [5]:
class CosmologicalModel_lambdaCDM:
    def __init__(self, z_min=0, z_max=1, z_step=256, max_ell=1000, omega_c=0.25, omega_b=0.05,
                 h=0.67, a_s=2.1e-9, n_s=0.96, m_nu=0.06):
        self.z_min = z_min
        self.z_max = z_max
        self.z_step = z_step
        self.max_ell = max_ell
        self.z = np.linspace(self.z_min, self.z_max, self.z_step)
        self.omega_c = omega_c
        self.omega_b = omega_b
        self.h = h
        self.a_s = a_s
        self.n_s = n_s
        self.m_nu = m_nu
        self.ells = np.arange(0, self.max_ell, 1)
        self.model = ccl.Cosmology(Omega_c=omega_c, Omega_b=omega_b, h=h, A_s=a_s, 
                                   n_s=n_s, m_nu=m_nu, mass_split="single", 
                                   transfer_function="boltzmann_camb")

    def generate_galaxy_clustering(self, nz_fn=default_nz, b1_fn=default_b1):
        nz = nz_fn(self.z)
        b1 = b1_fn(self.z)
        return ccl.NumberCountsTracer(self.model, has_rsd=False, dndz=(self.z, nz), bias=(self.z, b1))

    def generate_lensing(self, z_source=1100):
        return ccl.CMBLensingTracer(self.model, z_source=z_source)

    def generate_angular_ps(self, nz_fn=default_nz, b1_fn=default_b1, z_source=1100, output_type="all", is_plot=True):
        galaxy_dist = self.generate_galaxy_clustering(nz_fn, b1_fn)
        lensing_dist = self.generate_lensing(z_source=z_source)
        cgg = ccl.angular_cl(self.model, galaxy_dist, galaxy_dist, self.ells)
        ckk = ccl.angular_cl(self.model, lensing_dist, lensing_dist, self.ells)
        ckg = ccl.angular_cl(self.model, lensing_dist, galaxy_dist, self.ells)
        if output_type == "cgg":
            if is_plot:
                plt.loglog(self.ells, cgg, label="Galaxy-Galaxy")
                plt.xlabel("Angular Scale")
                plt.ylabel("Spectrum Density")
                plt.show()
            return cgg
        elif output_type == "ckg":
            if is_plot:
                plt.loglog(self.ells, ckg, label="Lensing-Galaxy")
                plt.xlabel("Angular Scale")
                plt.ylabel("Spectrum Density")
                plt.show()
            return ckg
        elif output_type == "ckk":
            if is_plot:
                plt.loglog(self.ells, ckk, label="Lensing-Lensing")
                plt.xlabel("Angular Scale")
                plt.ylabel("Spectrum Density")
                plt.show()
            return ckk
        else:
            if is_plot:
                plt.loglog(self.ells, ckg, label="Lensing-Galaxy")
                plt.loglog(self.ells, cgg, label="Galaxy-Galaxy")
                plt.loglog(self.ells, ckk, label="Lensing-Lensing")
                plt.xlabel("Angular Scale")
                plt.ylabel("Spectrum Density")
                plt.legend(loc="best")
                plt.show()
            return cgg, ckg, ckk

    def generate_survey_map(self, nz_fn=default_nz, b1_fn=default_b1, z_source=1100, nside=1024, pol=False, is_plot=True):
        np.random.seed(998905029)
        cl = self.generate_angular_ps(nz_fn=nz_fn, b1_fn=b1_fn, z_source=z_source, is_plot=False)
        delta_g, delta_k = hp.synfast(list(cl), nside=nside, pol=pol)
        if is_plot:
            hp.mollview(delta_g, cmap='seismic', min=-.1, max=.1, title="Galaxy Overdensity Field")
            hp.mollview(delta_k, cmap='seismic', min=-.1, max=.1, title="Lensing Overdensity Field")
            plt.show()
        return delta_g, delta_k

In [6]:
class CosmologicalModel_wCDM:
    def __init__(self, z_min=0, z_max=1, z_step=256, max_ell=1000, omega_c=0.25, omega_b=0.05,
                 h=0.67, a_s=2.1e-9, n_s=0.96, m_nu=0.06, w0=-1, wa=0):
        self.z_min = z_min
        self.z_max = z_max
        self.z_step = z_step
        self.max_ell = max_ell
        self.z = np.linspace(self.z_min, self.z_max, self.z_step)
        self.omega_c = omega_c
        self.omega_b = omega_b
        self.h = h
        self.a_s = a_s
        self.n_s = n_s
        self.m_nu = m_nu
        self.w0 = w0
        self.wa = wa
        self.ells = np.arange(0, self.max_ell, 1)
        self.model = ccl.Cosmology(Omega_c=omega_c, Omega_b=omega_b, h=h, A_s=a_s, 
                                   n_s=n_s, m_nu=m_nu, w0=w0, wa=wa, mass_split="single", 
                                   transfer_function="boltzmann_camb", 
                                   extra_parameters={"camb": {"dark_energy_model": "ppf"}})

    def generate_galaxy_clustering(self, nz_fn=default_nz, b1_fn=default_b1):
        nz = nz_fn(self.z)
        b1 = b1_fn(self.z)
        return ccl.NumberCountsTracer(self.model, has_rsd=False, dndz=(self.z, nz), bias=(self.z, b1))

    def generate_lensing(self, z_source=1100):
        return ccl.CMBLensingTracer(self.model, z_source=z_source)

    def generate_angular_ps(self, nz_fn=default_nz, b1_fn=default_b1, z_source=1100, output_type="all", is_plot=True):
        galaxy_dist = self.generate_galaxy_clustering(nz_fn, b1_fn)
        lensing_dist = self.generate_lensing(z_source=z_source)
        cgg = ccl.angular_cl(self.model, galaxy_dist, galaxy_dist, self.ells)
        ckk = ccl.angular_cl(self.model, lensing_dist, lensing_dist, self.ells)
        ckg = ccl.angular_cl(self.model, lensing_dist, galaxy_dist, self.ells)
        if output_type == "cgg":
            if is_plot:
                plt.loglog(self.ells, cgg, label="Galaxy-Galaxy")
                plt.xlabel("Angular Scale")
                plt.ylabel("Spectrum Density")
                plt.show()
            return cgg
        elif output_type == "ckg":
            if is_plot:
                plt.loglog(self.ells, ckg, label="Lensing-Galaxy")
                plt.xlabel("Angular Scale")
                plt.ylabel("Spectrum Density")
                plt.show()
            return ckg
        elif output_type == "ckk":
            if is_plot:
                plt.loglog(self.ells, ckk, label="Lensing-Lensing")
                plt.xlabel("Angular Scale")
                plt.ylabel("Spectrum Density")
                plt.show()
            return ckk
        else:
            if is_plot:
                plt.loglog(self.ells, ckg, label="Lensing-Galaxy")
                plt.loglog(self.ells, cgg, label="Galaxy-Galaxy")
                plt.loglog(self.ells, ckk, label="Lensing-Lensing")
                plt.xlabel("Angular Scale")
                plt.ylabel("Spectrum Density")
                plt.legend(loc="best")
                plt.show()
            return cgg, ckg, ckk

    def generate_survey_map(self, nz_fn=default_nz, b1_fn=default_b1, z_source=1100, nside=1024, pol=False, is_plot=True):
        np.random.seed(998905029)
        cl = self.generate_angular_ps(nz_fn=nz_fn, b1_fn=b1_fn, z_source=z_source, is_plot=False)
        delta_g, delta_k = hp.synfast(list(cl), nside=nside, pol=pol)
        if is_plot:
            hp.mollview(delta_g, cmap='seismic', min=-.1, max=.1, title="Galaxy Overdensity Field")
            hp.mollview(delta_k, cmap='seismic', min=-.1, max=.1, title="Lensing Overdensity Field")
            plt.show()
        return delta_g, delta_k

In [7]:
# parameters are obtained from https://arxiv.org/pdf/1807.06209 table 2 (TT,TE,EE+lowE+lensing+BAO)

# true distribution for parameters
true_h_dist_norm = stats.norm(loc=0.6766, scale=0.0042 * 1.5)
true_omega_b_dist_norm = stats.norm(loc=0.04897, scale=0.00053 * 1.5)
true_omega_c_dist_norm = stats.norm(loc=0.26066, scale=0.00302 * 1.5)
true_ns_dist_norm = stats.norm(loc=0.9665, scale=0.0038 * 1.5)
true_as_dist_norm = stats.norm(loc=2.105e-9, scale=0.030e-9 * 1.5)


# for neutrino mass, use upper bound = 1.1, lower bound = 0, mean = 0.6 and std = 0.6

# parameters
mu = 0.4  # mean
sigma = 0.4  # standard normal
lower_bound = 0
upper_bound = 0.8

# compute standardized bounds
a = (lower_bound - mu) / sigma
b = (upper_bound - mu) / sigma

true_mnu_dist_norm = stats.truncnorm(a, b, loc=mu, scale=sigma)


# for w0 and wa, we establish multivariate gaussian distribution
# from the paper https://arxiv.org/pdf/1807.06209 and table 6, 
# we use Planck+SNe+BAO for mean and variance

mean_w0 = -0.957
sigma_w0 = 0.080
mean_wa = -0.29
sigma_wa = 0.29

rho = -0.5

mean_mat = np.array([mean_w0, mean_wa])
cov_mat = np.array([[sigma_w0**2, rho * sigma_w0 * sigma_wa], [rho * sigma_w0 * sigma_wa, sigma_wa**2]])

true_w0_wa_dist_norm = stats.multivariate_normal(mean=mean_mat, cov=cov_mat)



In [8]:
# fix h and omega_b

# use h from DESI
# use omega_b from https://arxiv.org/abs/2401.15054

h = 0.6766
omega_b = 0.02218 / (h**2)
#h = true_param[0]
#omega_b = true_param[1]
omega_c = 0.256
#n_s = 0.975
n_s = 0.970
a_s = 2.176 * 10 ** -9
np.random.seed(998905029)
m_nu = 0.06
#m_nu = true_mnu_dist_norm.rvs(1)[0]
np.random.seed(99890502)
#w0, wa = true_w0_wa_dist_norm.rvs(1)
w0, wa = -0.9, -0.2

true_param = np.array([h, omega_b, omega_c, n_s, a_s, m_nu, w0, wa])

# true_param = np.array([h, omega_b, omega_c, n_s, a_s, m_nu])


np.save("true_parameters_w0_wa_-0.9_-0.2_1_3.npy", true_param)

In [9]:
N = 1000
min_ell = 75
max_ell = 1000
n = 10
# true_universe = CosmologicalModel_lambdaCDM(h=h, 
#                                   omega_b=omega_b, 
#                                   omega_c=omega_c, 
#                                   n_s=n_s, 
#                                   a_s=a_s, 
#                                   m_nu=m_nu)

true_universe = CosmologicalModel_wCDM(h=h, 
                                      omega_b=omega_b, 
                                      omega_c=omega_c, 
                                      n_s=n_s, 
                                      a_s=a_s, 
                                      m_nu=m_nu, 
                                      w0=w0, 
                                      wa=wa)

In [10]:
def compute_spectra(cgg, ckg, ckk, max_ell):
    delta_g, delta_k = hp.synfast([cgg, ckg, ckk], nside=1024, pol=False)
    measured_Cgg = hp.anafast(delta_g, delta_g, lmax=max_ell-1, pol=False, use_pixel_weights=True)[min_ell:]
    measured_Ckg = hp.anafast(delta_k, delta_g, lmax=max_ell-1, pol=False, use_pixel_weights=True)[min_ell:]
    measured_Ckk = hp.anafast(delta_k, delta_k, lmax=max_ell-1, pol=False, use_pixel_weights=True)[min_ell:]
    return measured_Cgg, measured_Ckg, measured_Ckk

cgg, ckg, ckk = true_universe.generate_angular_ps(is_plot=False)

start = time.time()

measured_Cgg_lst = np.zeros((N, max_ell - min_ell))
measured_Ckg_lst = np.zeros((N, max_ell - min_ell))
measured_Ckk_lst = np.zeros((N, max_ell - min_ell))

for i in range(N):
    result_cgg, result_ckg, result_ckk = compute_spectra(cgg, ckg, ckk, max_ell)
    measured_Cgg_lst[i] = result_cgg
    measured_Ckg_lst[i] = result_ckg
    measured_Ckk_lst[i] = result_ckk

_, galaxy_galaxy_bin, lensing_galaxy_bin, lensing_lensing_bin = create_bin(min_ell, max_ell, n + 1, 
                                                                                     cgg[min_ell:], 
                                                                                     ckg[min_ell:], 
                                                                                     ckk[min_ell:])


stacked_ps = np.hstack((galaxy_galaxy_bin, lensing_galaxy_bin, lensing_lensing_bin)).flatten()

cov = np.zeros((N, 3*n))
for i in range(N):
    _, cgg_bins, ckg_bins, ckk_bins = create_bin(min_ell, max_ell, n + 1, 
                                                 measured_Cgg_lst[i], 
                                                 measured_Ckg_lst[i], 
                                                 measured_Ckk_lst[i])
    cov[i, :] = np.hstack((cgg_bins, ckg_bins, ckk_bins)).flatten()

cov = np.cov(cov, rowvar=False)

np.save("cov_5_param__w0_wa_-0.9_-0.2_1_3.npy", cov)
np.save("stacked_ps_5_param__w0_wa_-0.9_-0.2_1_3.npy", stacked_ps)

end = time.time()
print(f"Constructing arrays for cov took {round(end - start, 1)} seconds")

Constructing arrays for cov took 2250.0 seconds
